In [1]:
import os
import pandas as pd
import numpy as np
import my_search
import xarray as xr
import cftime
import qgrid
xr.__version__

'0.13.0'

In [45]:
df = pd.read_csv('http://fletcher.ldeo.columbia.edu/catalogs/exceptions.csv')

In [46]:
df

,source_id,experiment_id,member_id,table_id,variable_id,grid_label,reason_code,reason_txt
0,EC-Earth3,historical,r2i1p1f1,Omon,tos,gr,noUse,half of files are gr- the other are still gn
1,EC-Earth3,historical,r2i1p1f1,Amon,all,gr,noUse,half of files are gr- the other are still gn
2,EC-Earth3,historical,r24i1p1f1,Amon,uas,gr,noUse,after 2008 the files do not concatenate
3,EC-Earth3,historical,r24i1p1f1,Amon,vas,gr,noUse,after 2008 the files do not concatenate
4,EC-Earth3,historical,r2i1p1f1,LImon,snw,gr,noUse,half of files are gr- the other are still gn
...,...,...,...,...,...,...,...,...
106,EC-Earth3-LR,piControl,r1i1p1f1,Amon,vas,gr,noUse,change Amon to Aclim
107,EC-Earth3-LR,piControl,r1i1p1f1,SImon,siconc,gn,noUse,change SImon to SIclim
108,EC-Earth3-LR,piControl,r1i1p1f1,SImon,sithick,gn,noUse,change SImon to SIclim
109,EC-Earth3-Veg,historical,r1i1p1f1,Omon,masscello,gn,noUse,change Omon to Oclim


In [47]:
cat = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6.csv')

In [65]:
# define a simple search on keywords
def search_df(df, **search):
    d = df
    for skey in search.keys():
        #print(skey,search[skey])
        d = d [ d[skey] == search[skey] ]
    return d

In [68]:
keys = ['source_id','experiment_id','member_id', 'table_id', 'variable_id', 'grid_label'] 
dfall = []
for index, row in df[0:].iterrows():
    drow = row.to_dict()
    del drow['reason_code']
    del drow['reason_txt']
    if drow['variable_id'] == 'all'
    dfall += [search_df(cat,**drow)]
dfcat = pd.concat(dfall,sort=False)
qgrid.show_grid(dfcat)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [1]:
# grab tracking id from a zarr:
tracking_id = "hdl:21.14100/dca6a9f1-0ab9-4ab4-a661-9c49955bbd99\nhdl:21.14100/c13d7735-561e-4d34-9a9e-ec587ad18a2f\nhdl:21.14100/27d2a2f1-4ef7-4dfd-b1aa-c912ce569054\nhdl:21.14100/b36801f6-359a-4ce4-a423-f89676444659\nhdl:21.14100/7d61d119-9c99-4afa-9376-cd016c6520b6\nhdl:21.14100/2ebfd9a7-dea1-4bb6-97c1-a636901ed2e1\nhdl:21.14100/c5563f39-a6c7-41c8-a6e4-9221859019c9\nhdl:21.14100/f55c6182-0bfb-45c5-bebb-98af2465f6f5\nhdl:21.14100/e5dd99a9-df00-4c9b-93b2-e3d831a16d4e"

In [4]:
tracks = tracking_id.split('\n')

In [ ]:
import my_search
import pandas as pd

df_list = []
for track in tracks:
    files= my_search.esgf_search(server="https://esgf-node.llnl.gov/esg-search/search", mip_era='CMIP6', tracking_id=track, 
                                     page_size=500, verbose=False)
    files.loc[:,'version'] = [str.split('/')[-2] for str in files['HTTPServer_url']]
    files.loc[:,'file_name'] = [str.split('/')[-1] for str in files['HTTPServer_url']]
    # might need to set activity_id to activity_drs for some files (see old versions)
    files.loc[:,'activity_id'] = files.activity_drs
    files.loc[:,'time_range'] = [str[-28:-3] for str in files['file_name']]
    files.loc[:,'century'] = [str[-28:-26] for str in files['file_name']]
    #if (experiment_id=='highresSST-present')&(('3hr' in table_id)|('6hr' in table_id)):
    #    files = files[files.century=='20'] 

    df_list += [files.drop_duplicates(subset =["file_name","version","checksum"]) ]
dESGF = pd.concat(df_list,sort=False)
dESGF = dESGF.drop_duplicates(subset =["file_name","version","checksum"])

In [13]:
dESGF.keys()

Index(['Globus_url', 'GridFTP_url', 'HTTPServer_url', 'OPENDAP_url',
       '_timestamp', '_version_', 'activity_drs', 'activity_id',
       'branch_method', 'cf_standard_name', 'checksum', 'checksum_type',
       'citation_url', 'data_node', 'data_specs_version', 'dataset_id',
       'dataset_id_template_', 'directory_format_template_', 'experiment_id',
       'experiment_title', 'frequency', 'further_info_url', 'grid',
       'grid_label', 'id', 'index_node', 'instance_id', 'institution_id',
       'latest', 'master_id', 'member_id', 'mip_era', 'model_cohort',
       'nominal_resolution', 'pid', 'product', 'project', 'realm', 'replica',
       'retracted', 'score', 'size', 'source_id', 'source_type',
       'sub_experiment_id', 'table_id', 'timestamp', 'title', 'tracking_id',
       'type', 'variable', 'variable_id', 'variable_long_name',
       'variable_units', 'variant_label', 'version', 'file_name', 'time_range',
       'century'],
      dtype='object')

In [19]:
dESGF.dataset_id.values

array(['CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r1i1p1f1.day.hur.gn.v20190603|aims3.llnl.gov',
       'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r1i1p1f1.day.hur.gn.v20190603|aims3.llnl.gov',
       'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r1i1p1f1.day.hur.gn.v20190603|aims3.llnl.gov',
       'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r1i1p1f1.day.hur.gn.v20190603|aims3.llnl.gov',
       'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r1i1p1f1.day.hur.gn.v20190603|aims3.llnl.gov',
       'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r1i1p1f1.day.hur.gn.v20190603|aims3.llnl.gov',
       'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r1i1p1f1.day.hur.gn.v20190603|aims3.llnl.gov',
       'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r1i1p1f1.day.hur.gn.v20190603|aims3.llnl.gov',
       'CMIP6.ScenarioMIP.MRI.MRI-ESM2-0.ssp245.r1i1p1f1.day.hur.gn.v20190603|aims3.llnl.gov'],
      dtype=object)

In [18]:
dESGF.time_range

0    p1f1_gn_20150101-20241231
0    p1f1_gn_20250101-20341231
0    p1f1_gn_20350101-20441231
0    p1f1_gn_20450101-20541231
0    p1f1_gn_20550101-20641231
0    p1f1_gn_20650101-20741231
0    p1f1_gn_20750101-20841231
0    p1f1_gn_20850101-20941231
0    p1f1_gn_20950101-21001231
Name: time_range, dtype: object

In [21]:
dESGF.HTTPServer_url.values

array(['http://aims3.llnl.gov/thredds/fileServer/css03_data/CMIP6/ScenarioMIP/MRI/MRI-ESM2-0/ssp245/r1i1p1f1/day/hur/gn/v20190603/hur_day_MRI-ESM2-0_ssp245_r1i1p1f1_gn_20150101-20241231.nc',
       'http://aims3.llnl.gov/thredds/fileServer/css03_data/CMIP6/ScenarioMIP/MRI/MRI-ESM2-0/ssp245/r1i1p1f1/day/hur/gn/v20190603/hur_day_MRI-ESM2-0_ssp245_r1i1p1f1_gn_20250101-20341231.nc',
       'http://aims3.llnl.gov/thredds/fileServer/css03_data/CMIP6/ScenarioMIP/MRI/MRI-ESM2-0/ssp245/r1i1p1f1/day/hur/gn/v20190603/hur_day_MRI-ESM2-0_ssp245_r1i1p1f1_gn_20350101-20441231.nc',
       'http://aims3.llnl.gov/thredds/fileServer/css03_data/CMIP6/ScenarioMIP/MRI/MRI-ESM2-0/ssp245/r1i1p1f1/day/hur/gn/v20190603/hur_day_MRI-ESM2-0_ssp245_r1i1p1f1_gn_20450101-20541231.nc',
       'http://aims3.llnl.gov/thredds/fileServer/css03_data/CMIP6/ScenarioMIP/MRI/MRI-ESM2-0/ssp245/r1i1p1f1/day/hur/gn/v20190603/hur_day_MRI-ESM2-0_ssp245_r1i1p1f1_gn_20550101-20641231.nc',
       'http://aims3.llnl.gov/thredds/fileSer

In [22]:
dESGF.retracted

0    False
0    False
0    False
0    False
0    False
0    False
0    False
0    False
0    False
Name: retracted, dtype: bool

In [23]:
dESGF.replica

0    True
0    True
0    True
0    True
0    True
0    True
0    True
0    True
0    True
Name: replica, dtype: bool